### Base NewsAPI interaction

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch
from tqdm import tqdm
import psycopg2

# API & DB Setup
NEWSAPI_KEY = "52613d226c4748a29934e6673b77606c"
DB_PARAMS = {
    'dbname': 'twt_snt',
    'user': 'postgres',
    'password': 'Ilpmnl!69gg',
    'host': 'localhost',
    'port': '5432'
}

# Term Mapping
term_mapping = {
    "Bitcoin": "BTC", "DogWifHat": "WIF", "Cardano": "ADA", "Avax": "AVAX",
    "Ethereum": "ETH", "Solana": "SOL", "Sui": "SUI", "Chainlink": "LINK", "Dogecoin": "DOGE",
    "Kaspa": "KAS", "Popcat": "POPCAT", "Helium Mobile": "HNT", "Polygon": "MATIC", "Aave": "AAVE", "Tesla": "TSLA",
    "JPMorgan": "JPM", "Chevron": "CVX", "Apple": "AAPL", "Google": "GOOGL", "Nvidia": "NVDA",
    "Amazon": "AMZN", "Gamestop": "GME", "Coinbase": "COIN", "Microsoft": "MSFT", "Disney": "DIS",
    "Taiwan Semi": "TSMC", "Netflix": "NFLX", "Hedara": "HBAR", "Pepe": "PEPE", "Ripple": "XRP"
}
keywords = list(term_mapping.keys())
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=3)).strftime('%Y-%m-%d')

# Sentiment Setup
sia = SentimentIntensityAnalyzer()
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base')

def softmax(x):
    return F.softmax(torch.tensor(x), dim=-1).numpy()

def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    return {'roberta_neg': scores[0], 'roberta_pos': scores[1]}

# Exclusions
excluded_sources = ["Unity.com", "Ozbargain.com", "Erickimphotography.com", "Slickdeals.net"]

def extract_source(url):
    if pd.isna(url):
        return None
    return url.split('//')[-1].split('/')[0].replace('www.', '').replace('.com', '').replace('.co', '').title()

# Fetch News
all_articles = []
for keyword in keywords:
    params = {
        "q": keyword,
        "from": start_date,
        "to": end_date,
        "sortBy": "relevancy",
        "language": "en",
        "apiKey": NEWSAPI_KEY
    }
    print(f"Fetching news articles for: {keyword}...")
    response = requests.get("https://newsapi.org/v2/everything", params=params)
    if response.status_code == 200:
        data = response.json()
        if "articles" in data:
            pulled_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            for article in data["articles"]:
                source_name = article["source"].get("name", "N/A")
                if source_name not in excluded_sources:
                    all_articles.append({
                        "search_keyword": keyword,
                        "term": term_mapping.get(keyword, keyword),
                        "title": article.get("title", "N/A"),
                        "summary": article.get("description", "N/A"),
                        "published_date": article.get("publishedAt", "N/A"),
                        "pulled_date": pulled_date,
                        "url": article.get("url", "N/A"),
                        "source": source_name
                    })

# DataFrame Creation
news_df = pd.DataFrame(all_articles)

# Filter
def filter_articles(row):
    summary_text = str(row["summary"]).lower() if pd.notna(row["summary"]) else ""
    return row["search_keyword"].lower() in summary_text or row["term"].lower() in summary_text

news_df = news_df[news_df.apply(filter_articles, axis=1)].reset_index(drop=True)

# Sentiment Analysis
res = {}
for i, row in tqdm(news_df.iterrows(), total=len(news_df)):
    summary_text = row['summary']
    article_id = row['url']
    if article_id in res:
        continue
    vader_result = sia.polarity_scores(summary_text)
    vader_result_rename = {f"vader_{key}": value for key, value in vader_result.items()}
    roberta_result = polarity_scores_roberta(summary_text)
    res[article_id] = {**vader_result_rename, **roberta_result}

sentiment_df = pd.DataFrame.from_dict(res, orient='index')
news_df = news_df.merge(sentiment_df, left_on='url', right_index=True)

# ✅ Assign `date` and extract clean source
news_df['date'] = pd.to_datetime(news_df['published_date'], errors='coerce').dt.strftime('%Y-%m-%d')
news_df['source'] = news_df['url'].apply(extract_source)

# --------------------------------------------
# ✅ Insert to PostgreSQL
# --------------------------------------------
def safe_value(val):
    return val if pd.notnull(val) else None

conn = psycopg2.connect(**DB_PARAMS)
cursor = conn.cursor()
insert_query = """
    INSERT INTO articles_tbl (title, summary, url, published_date, pulled_date, term, 
                              vader_neg, vader_neu, vader_pos, vader_compound, 
                              roberta_neg, roberta_pos, date, source)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (url) DO NOTHING
"""
batch_size = 40
batch_values = []

for _, row in news_df.iterrows():
    vals = (
        safe_value(row['title']), safe_value(row['summary']), safe_value(row['url']),
        safe_value(row['published_date']), safe_value(row['pulled_date']), safe_value(row['term']),
        safe_value(row['vader_neg']), safe_value(row['vader_neu']), safe_value(row['vader_pos']), safe_value(row['vader_compound']),
        safe_value(row['roberta_neg']), safe_value(row['roberta_pos']), safe_value(row['date']), safe_value(row['source'])
    )
    batch_values.append(vals)

    if len(batch_values) >= batch_size:
        try:
            cursor.executemany(insert_query, batch_values)
            conn.commit()
            print(f"Inserted {len(batch_values)} rows")
            batch_values = []
        except Exception as e:
            print(f"Error inserting batch: {e}")
            conn.rollback()

if batch_values:
    try:
        cursor.executemany(insert_query, batch_values)
        conn.commit()
        print(f"Inserted final batch of {len(batch_values)} rows")
    except Exception as e:
        print(f"Error inserting final batch: {e}")
        conn.rollback()

cursor.close()
conn.close()
print("✅ Done.")

C:\Users\shane\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model fro

Fetching news articles for: Bitcoin...
Fetching news articles for: DogWifHat...
Fetching news articles for: Cardano...
Fetching news articles for: Avax...
Fetching news articles for: Ethereum...
Fetching news articles for: Solana...
Fetching news articles for: Sui...
Fetching news articles for: Chainlink...
Fetching news articles for: Dogecoin...
Fetching news articles for: Kaspa...
Fetching news articles for: Popcat...
Fetching news articles for: Helium Mobile...
Fetching news articles for: Polygon...
Fetching news articles for: Aave...
Fetching news articles for: Tesla...
Fetching news articles for: JPMorgan...
Fetching news articles for: Chevron...
Fetching news articles for: Apple...
Fetching news articles for: Google...
Fetching news articles for: Nvidia...
Fetching news articles for: Amazon...
Fetching news articles for: Gamestop...
Fetching news articles for: Coinbase...
Fetching news articles for: Microsoft...
Fetching news articles for: Disney...
Fetching news articles for: Ta

100%|██████████████████████████████████████████████████████████████████████████████| 806/806 [1:56:40<00:00,  8.69s/it]


Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted 40 rows
Inserted final batch of 6 rows
✅ Done.


In [5]:
import psycopg2
import pandas as pd
from datetime import datetime

# ✅ Database connection parameters
db_params = {
    'dbname': 'twt_snt',
    'user': 'postgres',
    'password': 'Ilpmnl!69gg',
    'host': 'localhost',
    'port': '5432'
}

# ✅ Function to safely format values for SQL
def safe_value(value):
    if isinstance(value, str):
        return value
    elif isinstance(value, pd.Timestamp):  # Convert pandas datetime to string
        return value.strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(value) else None
    elif value is None:  # Explicitly handle None values
        return None
    else:
        return None if pd.isnull(value) else value

# ✅ Create a connection to the database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# ✅ Define the insert query with ON CONFLICT DO NOTHING (include 'source')
insert_query = """
    INSERT INTO articles_tbl (
        title, summary, url, published_date, pulled_date, term,
        vader_neg, vader_neu, vader_pos, vader_compound,
        roberta_neg, roberta_pos, source  -- ✅ Include 'source'
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (url) DO NOTHING
"""

batch_size = 40
batch_values = []

try:
    # ✅ Convert 'pulled_date' to datetime format
    df['pulled_date'] = pd.to_datetime(df['pulled_date'])

    for index, row in df.iterrows():
        # ✅ Prepare values for insertion (Ensuring correct column names)
        values = (
            safe_value(row['title']),  # ✅ Matches column name in df
            safe_value(row['summary']),
            safe_value(row['url']),
            safe_value(row['published_date']),
            safe_value(row['pulled_date']),
            safe_value(row['term']),
            safe_value(row['vader_neg']),
            safe_value(row['vader_neu']),
            safe_value(row['vader_pos']),
            safe_value(row['vader_compound']),
            safe_value(row['roberta_neg']),
            safe_value(row['roberta_pos']),
            safe_value(row['source'])  # ✅ Include 'source'
        )

        batch_values.append(values)

        # ✅ Commit in batches
        if len(batch_values) >= batch_size:
            try:
                cursor.executemany(insert_query, batch_values)
                conn.commit()
                print(f"✅ Inserted {len(batch_values)} rows at {datetime.now()}")
                batch_values = []  # ✅ Clear the batch list after committing
            except Exception as e:
                print(f"❌ Error inserting batch: {e}")
                conn.rollback()

    # ✅ Insert remaining rows
    if batch_values:
        try:
            cursor.executemany(insert_query, batch_values)
            conn.commit()
            print(f"✅ Inserted final batch of {len(batch_values)} rows at {datetime.now()}")
        except Exception as e:
            print(f"❌ Error inserting final batch: {e}")
            conn.rollback()

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    # ✅ Close the cursor and connection
    cursor.close()
    conn.close()

❌ Error: name 'df' is not defined


In [6]:
# Convert 'published_date' column to datetime format (ensure UTC timezone)
df['published_date'] = pd.to_datetime(df['published_date'], utc=True)

# Sort by term and published_date in descending order
df_sorted = df.sort_values(by=['term', 'published_date'], ascending=[True, False])

# Get the 3 most recent dates for each term
recent_dates_df = df_sorted.groupby('term').head(3)
recent_dates_df

NameError: name 'df' is not defined

In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import requests
import json
from datetime import datetime, timedelta
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch

# Initialize RoBERTa model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base')
def softmax(x):
    return F.softmax(torch.tensor(x), dim=-1).numpy()

# Function to run RoBERTa model on the entire dataset
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg': scores[0],
        'roberta_pos': scores[1]
    }
    return scores_dict

# Define the base URL for the Alpha Vantage API
BASE_URL = "https://www.alphavantage.co/query"
API_KEY = "ZSODAHB79R0QMY9M"

# Get current time and 'start date' time for the API request
start_date = (datetime.now() - timedelta(days=2)).strftime('%Y%m%dT%H%M')
today = datetime.now().strftime('%Y%m%dT%H%M')

# List of tuples containing term information for analysis
search_and_names = [
    ('CRYPTO:BTC', 'Bitcoin', 'BTC'),
    ('CRYPTO:WIF', 'Dogwifhat', 'WIF'),
    ('CRYPTO:ADA', 'Cardano', 'ADA'),
    ('CRYPTO:AVAX', 'Avalanche', 'AVAX'),    
    ('CRYPTO:ETH', 'Ethereum', 'ETH'),
    ('CRYPTO:SOL', 'Solana', 'SOL'),
    ('CRYPTO:KAS', 'Kaspa', 'KAS'),
    ('CRYPTO:SUI', 'Sui', 'SUI'),
    ('CRYPTO:LINK', 'Chainlink', 'LINK'),
    ('CRYPTO:DOGE', 'Dogecoin', 'DOGE'),
    ('CRYPTO:POPCAT', 'Popcat', 'POPCAT'),
    ('CRYPTO:HNT', 'Helium', 'HNT'),
    ('CRYPTO:MATIC', 'Polygon', 'MATIC'),
    ('TSLA', 'Tesla', 'TSLA'),
    ('CVX', 'Chevron', 'CVX'),
    ('AAPL', 'Apple', 'AAPL'),
    ('GOOGL', 'Google', 'GOOGL'),
    ('MSFT', 'Microsoft', 'MSFT'),
    ('NVDA', 'Nvidia', 'NVDA'),
    ('GME', 'Gamestop', 'GME'),
    ('TSMC', 'Taiwan Semi', 'TSMC'),
    ('JPM', 'Morgan Chase', 'JPM'),
    ('AMZN', 'Amazon', 'AMZN'),
    ('COIN', 'Coinbase', 'COIN'),
    ('NFLX', 'Netflix', 'NFLX'),
    ('DIS', 'Disney', 'DIS'),
    ('DXY', 'DXY', 'DXY')
    
]

# Initialize an empty DataFrame to collect all results
all_articles_df = pd.DataFrame()

# Loop over each term in the list
# Loop over each term in the list
for term, name_in_title, new_term in search_and_names:
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": term,
        "time_from": start_date,
        "time_to": today,
        "sort": "RELEVANCE",
        "apikey": API_KEY
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        print(f"API response for {term}: {data}")  # Log the API response
        if "feed" in data:
            articles_list = []
            pulled_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            for article in data["feed"]:
                article_url = article.get("url", "N/A")
                title = article.get("title", "N/A")
                summary = article.get("summary", "N/A")

                # Log articles that don't meet the criteria
                if not ((name_in_title.lower() in title.lower() and name_in_title.lower() in summary.lower()) or
                        (new_term.lower() in title.lower() and new_term.lower() in summary.lower())):
                    print(f"Filtered out article: {title} (URL: {article_url})")

                # Check if the DataFrame is empty or if the URL already exists
                if all_articles_df.empty or article_url not in all_articles_df['URL'].values:
                    # Include the item if either name_in_title or new_term is in both title and summary
                    if ((name_in_title.lower() in title.lower() and name_in_title.lower() in summary.lower()) or
                        (new_term.lower() in title.lower() and new_term.lower() in summary.lower())):
                        article_details = {
                            "Title": title,
                            "Summary": summary,
                            "URL": article_url,
                            "Published Date": article.get("time_published", "N/A"),
                            "Pulled Date": pulled_date,
                            "Term": new_term
                        }
                        articles_list.append(article_details)
                else:
                    print(f"Skipping duplicate article: {article_url}")

            articles_df = pd.DataFrame(articles_list)

            if not articles_df.empty:
                # Initialize VADER sentiment analyzer
                sia = SentimentIntensityAnalyzer()
                res = {}

                for i, row in tqdm(articles_df.iterrows(), total=len(articles_df)):
                    try:
                        summary_text = row['Summary']
                        article_id = row['URL']

                        if article_id in res:
                            continue

                        vader_result = sia.polarity_scores(summary_text)
                        vader_result_rename = {f"vader_{key}": value for key, value in vader_result.items()}
                        roberta_result = polarity_scores_roberta(summary_text)
                        both_results = {**vader_result_rename, **roberta_result}
                        res[article_id] = both_results

                    except RuntimeError as e:
                        print(f'Error processing article with ID {article_id}: {e}')

                sentiment_df = pd.DataFrame.from_dict(res, orient='index')
                articles_snt_df = articles_df.merge(sentiment_df, left_on='URL', right_index=True)

                all_articles_df = pd.concat([all_articles_df, articles_snt_df], ignore_index=True)
                print(f"articles found for {term}.")
        else:
            print(f"No news articles found for {term}.")
    else:
        print(f"Failed to retrieve data for {term}. HTTP Status Code: {response.status_code}")
        
        
# commit `all_articles_df` to database below.

import psycopg2
import pandas as pd
from datetime import datetime

# Database connection parameters
db_params = {
    'dbname': 'twt_snt',
    'user': 'postgres',
    'password': 'Ilpmnl!69gg',
    'host': 'localhost',
    'port': '5432'
}

# Function to ensure data types are compatible with SQL
def safe_value(value):
    if isinstance(value, str):
        return value
    elif isinstance(value, pd.Timestamp):  # Convert pandas datetime to string
        return value.strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(value) else None
    elif value is None:  # Explicitly handle None values
        return None
    else:
        return None if pd.isnull(value) else value

# Create a connection to the database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Define the insert query with ON CONFLICT DO NOTHING
insert_query = """
    INSERT INTO articles_tbl (title, summary, url, published_date, pulled_date, term,
                              vader_neg, vader_neu, vader_pos, vader_compound,
                              roberta_neg, roberta_pos)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (url) DO NOTHING
"""

batch_size = 40
batch_values = []

try:
    # Ensure 'Pulled Date' is in datetime format
    all_articles_df['Pulled Date'] = pd.to_datetime(all_articles_df['Pulled Date'])

    for index, row in all_articles_df.iterrows():
        # Prepare values for insertion
        values = (
            safe_value(row['Title']),
            safe_value(row['Summary']),
            safe_value(row['URL']),
            safe_value(row['Published Date']),
            safe_value(row['Pulled Date']),
            safe_value(row['Term']),
            safe_value(row['vader_neg']),
            safe_value(row['vader_neu']),
            safe_value(row['vader_pos']),
            safe_value(row['vader_compound']),
            safe_value(row['roberta_neg']),
            safe_value(row['roberta_pos'])
        )

        batch_values.append(values)

        # Commit in batches
        if len(batch_values) >= batch_size:
            try:
                cursor.executemany(insert_query, batch_values)
                conn.commit()
                print(f"Inserted {len(batch_values)} rows at {datetime.now()}")
                batch_values = []  # Clear the batch list
            except Exception as e:
                print(f"Error inserting batch: {e}")
                conn.rollback()

    # Insert remaining rows
    if batch_values:
        try:
            cursor.executemany(insert_query, batch_values)
            conn.commit()
            print(f"Inserted final batch of {len(batch_values)} rows at {datetime.now()}")
        except Exception as e:
            print(f"Error inserting final batch: {e}")
            conn.rollback()

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()



C:\Users\shane\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model fro

API response for CRYPTO:BTC: {'items': '45', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'Bitcoin advocate Max keiser casts doubt over new BTC treasury companies', 'url': 'https://cointelegraph.com/news/max-keiser-casts-doubt-new-bitcoin-treasury-companies', 'time_published': '20250531T214742', 'authors': ['Vince Quill'], 'summary': 'Newer Bitcoin treasury companies have not yet been battle-tested in prolonged bear market conditions, the Bitcoin maximalist said.', 'banner_image': 'https://s3.cointelegraph.com/uploads/2025-05/0197284b-ac4a-7bd1-8373-aee5eaa8006d', 'source': 'Cointelegraph', 'category_within_source': 'n/a', 'source_domain': 'cointelegraph.com', 'topics': [{'topic': 'Technology', 'relevance_score': '1.0'}, {'topic': 'Financial Market

  0%|          | 0/24 [00:00<?, ?it/s]

articles found for CRYPTO:BTC.
API response for CRYPTO:WIF: {'Information': 'Invalid inputs. Please refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}
No news articles found for CRYPTO:WIF.
API response for CRYPTO:ADA: {'items': '2', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'XRP News: XenDex Presale Ends In 12 Hours, Join Presale Before $XDX Exchange Listing', 'url': 'https://www.benzinga.com/pressreleases/25/05/g45710565/xrp-news-xendex-presale-ends-in-12-hours-join-presale-before-xdx-exchange-listing', 'time_published': '20250531T164822', 'authors': ['Globe Newswire'], 'summary': 'SYDNEY, May 31, 2025 ( GLOBE NEWSWIRE ) -- With only 12 hours remaining, the clock is ticking for investors to secure

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

articles found for NVDA.
API response for GME: {'items': '2', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'This Week in Crypto Games: GameStop Buys Bitcoin, More Ethereum Games Close Up Shop', 'url': 'https://decrypt.co/323037/this-week-crypto-games-gamestop-bitcoin-ethereum-offline', 'time_published': '20250601T222557', 'authors': ['Andrew Hayward'], 'summary': 'Video game retailer GameStop makes its first big Bitcoin purchase, even more crypto games go offline, and other news from the last week.', 'banner_image': 'https://cdn.decrypt.co/resize/1024/height/512/wp-content/uploads/2025/05/gamestop-logo-decrypt-style-01-gID_7.jpg', 'source': 'Decrypt.co', 'category_within_source': 'n/a', 'source_domain': 'decrypt.co', 'topics': [{'topic': 'Retail & 

  0%|          | 0/4 [00:00<?, ?it/s]

articles found for AMZN.
API response for COIN: {'items': '10', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'An Extremely Profitable Business Plans to Go Public. What Investors Need to Know.', 'url': 'https://www.fool.com/investing/2025/05/31/an-extremely-profitable-business-plans-to-go-publi/', 'time_published': '20250531T094500', 'authors': ['Jon Quast'], 'summary': "On May 27, Circle Internet Group updated its filings to become a publicly traded company. It's possible that investors haven't heard of Circle. But cryptocurrency enthusiasts are likely quite familiar with Circle's stablecoin USD Coin ( CRYPTO: USDC ) , the second-largest stablecoin by market ...", 'banner_image': 'https://g.foolcdn.com/image/?url=https%3A%2F%2Fg.foolcdn.com%2Fedito

API response for NFLX: {'items': '1', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': "Consumer Tech News  ( May 26-May 30 ) : Germany May Impose 10% Online Platform Tax, Tesla's European Sales Slump & More - Apple  ( NASDAQ:AAPL ) , BYD  ( OTC:BYDDF ) ", 'url': 'https://www.benzinga.com/markets/large-cap/25/06/45712446/consumer-tech-news-may-26-may-30-germany-may-impose-10-online-platform-tax-teslas-european-sale', 'time_published': '20250601T172552', 'authors': ['Lekha Gupta'], 'summary': 'Germany is reportedly considering a 10% tax on major online platforms like Alphabet and Meta,which could escalate trade tensions with the US Apple is preparing to rebrand its software platforms using a year-based naming system ( e.g., iOS 26 instead of iOS 19 ) . 